In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.functional import interpolate
import numpy as np
import numpy.random as npr
from tqdm import trange
import sklearn.metrics as met
from sklearn.preprocessing import MinMaxScaler 

In [2]:
## ----------------------------------- MODEL CLASS AND FUNCTIONS ----------------------------------- ##

In [3]:
# ----------- MODEL CLASS ------------- #

# define cnn class
class cnn(nn.Module):
    def __init__(self, numChannels, classes):
        # call the parent constructor
        super(cnn, self).__init__()
        # initialize first set of CONV => POOL => RELU layers
        self.conv1 = nn.Conv2d(in_channels=numChannels, out_channels=200,
            kernel_size=(5,5), stride=2, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3)
        self.relu1 = nn.ReLU(inplace=True)
        
        # initialize second set of CONV => POOL => RELU layers
        self.dropout1 = nn.Dropout(0.6)
        self.conv2 = nn.Conv2d(in_channels=200, out_channels=650,
            kernel_size=(5,5), padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3)
        self.relu2 = nn.ReLU(inplace=True)
                               
        # initialize final set of CONV => POOL => RELU layers
        self.dropout2 = nn.Dropout(0.6)
        self.conv3 = nn.Conv2d(in_channels=650, out_channels=350,
            kernel_size=(5,5), padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3)
        self.relu3 = nn.ReLU(inplace=True)
                               
        # initialize our softmax classifier
        self.dropout3 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features=1400, out_features=128)
        self.relu4 = nn.ReLU(inplace=True)
            
        self.dropout4 = nn.Dropout(0.6)
        self.fc3 = nn.Linear(in_features=128, out_features=classes)
        self.logSoftmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        # pass the output from the previous layer through the second
        # set of CONV => RELU => POOL layers
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)                       
        
        # through final CONV => RELU => POOL layer
        x = self.dropout2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)
        
        # pass the output to our softmax classifier to get our output
        # predictions
        m = nn.Flatten()
        x = m(x)
        x = self.dropout3(x)
        x = self.fc2(x)
        x = self.relu4(x)
        x = self.dropout4(x)
        x = self.fc3(x)
        output = self.logSoftmax(x)
        # return the output predictions
        return output

In [4]:
# ----------- FUNCTIONS ------------- #

# function to reshape images
def change_dataset(changed, dataset, BS):
    ind = min(len(dataset),BS)
    X = np.zeros((ind,300,300,3))
    
    if ind != len(dataset):
        X = dataset[:ind]
        X = X.reshape((ind,300,300,3))
        res = np.zeros((X.shape[0],3,300,300))
        for i in range(X.shape[0]):
            res[i] = np.moveaxis(X[i],-1,0)
        res = interpolate(torch.tensor(res),size=(175,175),mode='bicubic',align_corners=False)
        
        if len(changed) == 1:
            changed = np.vstack([changed,np.array(res)])
            changed = changed [1:]
        else:
            changed = np.vstack([changed,np.array(res)])
        dataset = dataset[ind:]
        
    else:
        X = dataset
        X = X.reshape((ind,300,300,3))
        res = np.zeros((X.shape[0],3,300,300))
        for i in range(X.shape[0]):
            res[i] = np.moveaxis(X[i],-1,0)
        res = interpolate(torch.tensor(res),size=(175,175),mode='bicubic',align_corners=False)
        changed = np.vstack([changed,np.array(res)])
        dataset = np.array([])
    return changed, dataset

# testing function
def test(model,X_test,t_test,device='cpu',BS=128):
    model.to(device)
    # prep data
    X_test = torch.tensor(X_test, dtype=torch.float32)
    t_test = torch.tensor(t_test, dtype=torch.long)
    test_data = TensorDataset(X_test, t_test)

    X_test = []
    loader_test = DataLoader(test_data, batch_size=BS)
    
    # test
    y_preds = []
    model.eval()

    with torch.no_grad():
        for _, (x,y) in enumerate(loader_test):
            preds = []
            x=x.to(device)
            y=y.to(device)
            # make the predictions
            pred = model(x)
            preds += pred.argmax(1).cpu()
            y_preds += preds
            
            
    # compute accuracy and confusion matrix
    test_acc = np.sum(np.array(y_preds)==np.array(t_test))/len(t_test)
        
    model.to(torch.device('cpu'))
    return test_acc, y_preds

In [5]:
## ----------------------------------- MAIN ----------------------------------- ##

In [6]:
# ----------- DATA PATHS ------------- #
test_data_path = '../share/data_train.npy'
test_labels_path = '../share/labels_train.npy'

In [7]:
# ----------- PREPARING DATA ------------- #
# loading Data
dataset = np.load(test_data_path)
t_test = np.load(test_labels_path)
print(f'1. dataset.shape, labels.shape: {dataset.shape, t_test.shape}')

# fixing mislabeled data
location = [880, 165, 558, 2396, 3198, 3715, 3734, 3824, 4412, 5238, 38, 2303, 3127, 3467, 3515, 5404, 6178, 248, 827, 1139, 1799, 2491, 2546, 4533, 6024, 103, 392, 846, 913, 1487, 2971, 3234, 3851, 4008, 4405, 4442, 4952, 5348, 204, 272, 297, 1177, 1198, 1326, 1655, 2156, 2690, 2864, 3450, 5095, 5111, 5186, 5243, 5785, 6106,387, 3973, 5112, 5331, 5002, 5263, 6046, 1843, 2941, 3419, 4158, 4166, 4369, 4980, 689, 973, 3066, 3028, 4905, 5107, 5708, 5817, 357, 368, 1065, 2093, 3229, 3341, 4469, 5257, 5711]
true_label = [6, 4, 7, 9, 5, 6, 3, 8, 2, 0, 5, 1, 4, 6, 3, 7, 9, 2, 7, 2, 8, 5, 1, 0, 6, 5, 2, 5, 6, 0, 5, 5, 5, 8, 5, 2, 5, 1, 4, 4, 4, 1, 4, 7, 0, 4, 4, 4, 9, 4, 4, 4, 8, 4, 0, 8, 3, 5, 2, 5, 0, 4, 6, 8, 0, 6, 9, 5, 2, 0, 3, 6, 7, 2, 5, 7, 1, 2, 7, 0, 5, 8,3, 1, 6, 8]

for k in range(len(location)):
    t_test[location[k]] = true_label[k]

# downsample data
dataset = dataset.T
X_test = np.zeros((1,3,175,175))
while len(dataset) != 0:
    X_test, dataset = change_dataset(X_test,dataset,64)
print(f'2. X_test.shape, t_test.shape: {X_test.shape, t_test.shape}')

# min-max scaling
X_test = X_test.reshape((len(X_test), 3*175*175))
mms = MinMaxScaler()
X_test = mms.fit_transform(X_test)
X_test = X_test.reshape((len(X_test),3,175,175))
print(f'3. X_test.shape, t_test.shape: {X_test.shape, t_test.shape}')

1. dataset.shape, labels.shape: ((270000, 6195), (6195,))
2. X_test.shape, t_test.shape: ((6195, 3, 175, 175), (6195,))
3. X_test.shape, t_test.shape: ((6195, 3, 175, 175), (6195,))


In [8]:
# ----------- SET UP DEVICE AND MODEL ------------- #

# setup device to be cuda if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# initialize the model
model = cnn(numChannels=3,classes=10)

# load model
model.load_state_dict(torch.load('maib_model.h5'))

<All keys matched successfully>

In [9]:
# ----------- TESTING ------------- #

# testing
acc, y_preds = test(model,X_test,t_test,device)

# confustion matrix for test set.
con_mat = met.confusion_matrix(t_test, y_preds)

print(f'Test accuracy: {acc}\n\nTest confusion matrix:\n {con_mat}')

Test accuracy: 0.9937046004842615

Test confusion matrix:
 [[622   1   1   0   0   1   0   0   0   0]
 [  0 608   0   0   0   2   0   0   0   0]
 [  1   1 613   0   1   1   1   0   0   0]
 [  0   0   0 612   1   0   0   0   0   0]
 [  0   0   0   0 613   1   0   4   6   1]
 [  0   0   0   0   0 616   0   0   0   2]
 [  2   1   0   0   0   0 617   1   0   1]
 [  0   0   0   0   3   0   0 620   0   0]
 [  0   0   0   1   0   0   0   0 626   0]
 [  0   0   0   0   0   3   1   1   0 609]]
